In [3]:
pip install datasets


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
from datasets import load_dataset



In [2]:
from huggingface_hub import login
login()

In [69]:
# 1. تحميل بيانات AG News
import requests
import pandas as pd

# اخذ الداتا من API
train_url = "https://datasets-server.huggingface.co/rows?dataset=wangrongsheng%2Fag_news&config=default&split=train&offset=0"
test_url = "https://datasets-server.huggingface.co/rows?dataset=wangrongsheng%2Fag_news&config=default&split=test&offset=0"

train_response = requests.get(train_url)
train_data = train_response.json()["rows"]
train_df = pd.DataFrame([row["row"] for row in train_data])

test_response = requests.get(test_url)
test_data = test_response.json()["rows"]
test_df = pd.DataFrame([row["row"] for row in test_data])

df = pd.concat([train_df, test_df], ignore_index=True)
df

,text,label
0,Wall St. Bears Claw Back Into the Black (Reute...,2
1,Carlyle Looks Toward Commercial Aerospace (Reu...,2
2,Oil and Economy Cloud Stocks' Outlook (Reuters...,2
3,Iraq Halts Oil Exports from Main Southern Pipe...,2
4,"Oil prices soar to all-time record, posing new...",2
...,...,...
195,Eriksson doesn #39;t feel any extra pressure f...,1
196,Injured Heskey to miss England friendly NEWCAS...,1
197,"Staples Profit Up, to Enter China Market NEW ...",2
198,Delegation Is Delayed Before Reaching Najaf AG...,0


In [70]:
label_counts = df['label'].value_counts()
label_counts

label
2    90
3    59
0    30
1    21
Name: count, dtype: int64

In [111]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer  # تحويل النصوص إلى ميزات عددية
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report

In [112]:
def preprocess_text(text):
    """تنظيف النصوص عبر تحويلها إلى حروف صغيرة فقط."""
    return text.lower()

In [113]:
# تطبيق المعالجة المسبقة على النصوص
df["text"] = df["text"].apply(preprocess_text)

In [114]:
df

,text,label
0,wall st. bears claw back into the black (reute...,2
1,carlyle looks toward commercial aerospace (reu...,2
2,oil and economy cloud stocks' outlook (reuters...,2
3,iraq halts oil exports from main southern pipe...,2
4,"oil prices soar to all-time record, posing new...",2
...,...,...
195,eriksson doesn #39;t feel any extra pressure f...,1
196,injured heskey to miss england friendly newcas...,1
197,"staples profit up, to enter china market new ...",2
198,delegation is delayed before reaching najaf ag...,0


In [115]:
# 2. تقسيم البيانات إلى train and test
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)


In [122]:
# 3. convert text into Numerical features usnig TF-IDF
vectorizer = TfidfVectorizer(min_df=2,max_df=0.9)  
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [123]:
# 4. تدريب عدة models ومقارنتها
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "AdaBoost": AdaBoostClassifier(),
}

In [124]:
# تخزين نتائج الأداء
results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train_tfidf, y_train)  # تدريب النموذج
    y_pred = model.predict(X_test_tfidf)  # إجراء التنبؤات
    accuracy = accuracy_score(y_test, y_pred)  # حساب الدقة
    results[name] = accuracy
    print(f"{name} Accuracy: {accuracy:.5f}\n")
    print(classification_report(y_test, y_pred))

Training Logistic Regression...
Logistic Regression Accuracy: 0.57500

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.00      0.00      0.00         2
           2       0.55      0.95      0.69        19
           3       0.71      0.42      0.53        12

    accuracy                           0.57        40
   macro avg       0.31      0.34      0.30        40
weighted avg       0.47      0.57      0.49        40

Training Decision Tree...
Decision Tree Accuracy: 0.57500

              precision    recall  f1-score   support

           0       0.50      0.29      0.36         7
           1       0.25      0.50      0.33         2
           2       0.74      0.74      0.74        19
           3       0.46      0.50      0.48        12

    accuracy                           0.57        40
   macro avg       0.49      0.51      0.48        40
weighted avg       0.59      0.57      0.57        40



C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\WPy64-31220\python-3.12.2.amd64\Lib\site-packages\sklearn\metrics\_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\WPy64-3

In [125]:
# 5. عرض النموذج الأفضل
best_model = max(results, key=results.get)
print(f"Best Model: {best_model} with Accuracy: {results[best_model]:.4f}")

Best Model: KNN with Accuracy: 0.7000
